# Pre-processing and clustering 3k PBMCs with scanpy

We document the pre-processing and clustering of 3000 PBMCs from a helathy data with the help of the scanpy package.

In [2]:
## Data download. If you run the script for the first time, uncomment the three lines below by removing #
## Because the Jupyter notebook recognizes lines that start with ! as command-line commands
# !mkdir data
# !wget http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz -O data/pbmc3k_filtered_gene_bc_matrices.tar.gz
# !cd data; tar -xzf pbmc3k_filtered_gene_bc_matrices.tar.gz; rm !$

--2020-05-13 09:25:42--  http://cf.10xgenomics.com/samples/cell-exp/1.1.0/pbmc3k/pbmc3k_filtered_gene_bc_matrices.tar.gz
Resolving cf.10xgenomics.com (cf.10xgenomics.com)... 13.224.102.50, 13.224.102.63, 13.224.102.80, ...
Connecting to cf.10xgenomics.com (cf.10xgenomics.com)|13.224.102.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7621991 (7.3M) [application/x-tar]
Saving to: ‘data/pbmc3k_filtered_gene_bc_matrices.tar.gz’

data/pbmc3k_filtere 100%[===================>]   7.27M  5.17MB/s    in 1.4s    

2020-05-13 09:25:44 (5.17 MB/s) - ‘data/pbmc3k_filtered_gene_bc_matrices.tar.gz’ saved [7621991/7621991]

